In [1]:
import pandas as pd
import ads
import requests
import PyPDF2
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os

https://ui.adsabs.harvard.edu/help/api/api-docs.html#get-/resolver/-bibcode-/-link_type-
    


In [2]:
ADS_DEV_KEY = "A0QQKEjS3qBtL6389PMeohI3FTLIRqtB8bZwXc6Y"
ads.config.token = "A0QQKEjS3qBtL6389PMeohI3FTLIRqtB8bZwXc6Y"

In [3]:
bibcodes = pd.read_csv("bibcodes.csv")
bibcodes

,Bibcodes
0,1997HIP...C......0E
1,2003A&A...401..781F
2,2003AJ....126.1090C
3,2004ApJS..154...10F
4,2004ApJS..154...25R
...,...
2775,2022A&A...663A.145H
2776,2022ApJS..259...62I
2777,2022AJ....164..198M
2778,2021AJ....161....5W


In [4]:
# file = open('.//data//EA papers2//1886BD....C......0A.pdf', 'rb')
# reader = PyPDF2.PdfFileReader(file)
# page = reader.pages[0]
# print(page.extract_text())

In [5]:
def download_pub_pdf(bibcode, source): # source is "pub" or "eprint"
    headers = {
        'Authorization': f'Bearer {ADS_DEV_KEY}',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
    }
    session = requests.Session()
    retry = Retry(connect=2, backoff_factor=1)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    response = session.get(f'https://ui.adsabs.harvard.edu/link_gateway/{bibcode}/{source}_pdf', headers=headers)
    with open(f'.//data//EA papers2/{bibcode}.pdf', 'wb') as f:
        f.write(response.content)


previously, 2093 bad pdfs and 669 good pdfs

In [ ]:
directory = ".//data//EA papers2//"
files = os.listdir(directory)
downloaded_bibs = [f[:-4] for f in files]

for bibcode in bibcodes["Bibcodes"]:
    if bibcode in downloaded_bibs:
        file_name = bibcode + ".pdf"
        file_size = os.path.getsize(directory+file_name)
        
        if file_size < 100000:
            try:
                download_pub_pdf(bibcode, "pub")
            except:
                print(bibcode, 1)
                continue
        
#         file_size = os.path.getsize(directory+file_name)
#         if file_size < 100000:
#             try:
#                 download_pub_pdf(bibcode, "eprint")
#             except:
#                 print(bibcode, 2)
#                 continue
        file_size = os.path.getsize(directory+file_name)
        if file_size < 100000:
            try:
                download_pub_pdf(bibcode, "author")    
            except:
                print(bibcode, 3)
                continue
        
    else:
        download_pub_pdf(bibcode, "pub")
        
        file_size = os.path.getsize(directory+file_name)
        if file_size < 100000:
            try:
                download_pub_pdf(bibcode, "eprint")
            except:
                print(bibcode, 4)
                continue
        file_size = os.path.getsize(directory+file_name)
        if file_size < 100000:
            try:
                download_pub_pdf(bibcode, "author")
            except:
                print(bibcode, 5)
                continue


In [6]:
directory = ".//data//EA papers2//"
files = os.listdir(directory)
downloaded_bibs = [f[:-4] for f in files]

for bibcode in bibcodes["Bibcodes"]:
    if bibcode in downloaded_bibs:
        file_name = bibcode + ".pdf"
        file_size = os.path.getsize(directory+file_name)
        
        if file_size < 100000:
            print(bibcode)
            download_pub_pdf(bibcode, "ads")
        
    else:
        download_pub_pdf(bibcode, "pub")
        
        file_size = os.path.getsize(directory+file_name)
        if file_size < 100000:
            print(bibcode)
            download_pub_pdf(bibcode, "ads")




1997HIP...C......0E
1992AJ....104.1650C
1998A&A...333..231B
2003tmc..book.....C
2004ApJS..152..261W
1996AJ....111..439H
1991ApJS...76..383D
1989ApJS...71..245K
1980SAAOC...1..166C
1980SAAOC...1..234C
1994A&AS..104..429O
1994A&AS..106..257O
1993A&AS..102...89O
1966MNRAS.133..475A
1979ApJS...39...89E
1992AJ....103.2048G
1975MSS...C01....0H
1978MSS...C02....0H
1982MSS...C03....0H
1988MSS...C04....0H
1999mctd.book.....H
2000PASP..112..961B
2000A&A...355L..27H
2000AcA....50..509G
1979PASP...91..589B
1996ApJ...469..355F
1978bsch.book.....B
2000AJ....119.1448H
1980afcp.book.....L
2005PrvComC01....0C
1993Icar..101..108K
2005PThPS.158...24M
1932ADS...C......0A
1903BD....C......0A
1892RNAO...16....1T
1896AnCap...3....1G
1983BICDS..24...83D
1776QB65.F5........
1988VeARI..32....1F
1971LowPM.C......0G
1991adc..rept.....G
2001AJ....121.1752M
1918AnHar..91....1C
1992BICDS..41....9T
1964BS....C......OH
1979LHS...C......0L
1941POMin...3...33L
1941POMin...3....1L
1957LTT...C......0L
1952GCTP..C......0J


2002ApJ...564..466G
2002ApJ...564L..89H
2002ApJ...567L..59G
2002ApJ...567L.133P
2002ApJ...568L.107L
2002ApJ...571..519L
2002ApJ...578..536Z
2003AJ....125..343L
2003AJ....125..850B
2003AJ....125.3302G
2003AJ....126..975T
2003AJ....126.1526B
2003AJ....126.2421C
2003AJ....126.2487B
2003ApJ...586L.149S
2003ApJ...591L..49L
2003ApJ...592.1186B
2003ApJ...594..510B
2003IAUJD...8E..60B
2003IAUS..211..197W
2003PASP..115.1207T
2004AJ....127.2856B
2004AJ....127.2948V
2004AJ....127.3553K
2004AJ....128.1733G
2004ApJ...600.1020M
2004ApJ...604L..61C
2010ApJ...719..497L
2010ApJ...719..890R
2010ApJ...717..348H
1999A&A...349..389V
2012AJ....143...81S
2011ApJ...726...52H
2010ApJ...721L.153J
2011AJ....141...16J
2012ApJ...749..134H
2010ApJ...725.2017K
2012ApJ...745...80Q
2011AJ....141....8S
2010Sci...330...51H
2011ApJ...727...24T
2010PASP..122.1465M
2011ApJ...728..138H
2011ApJ...726L..19A
2011ApJ...728..120M
2010ApJ...725.1633H
2010AJ....140.1657M
2011ApJ...730...10H
2011ApJ...726..113I
1990AJ.....99.2019L


2012MNRAS.422L..24Q
2012ApJ...755..102Y
2013ApJ...777....3N
2014MNRAS.437..475M
2013Sci...342..331H
2013ApJ...768...14W
2013ApJ...778..185M
2013ApJ...779L..26R
2014ApJ...781...18C
2013Natur.503..377P
2014ApJ...780...54B
2013ApJ...779L..23P
2013ApJ...778...38H
2014ApJ...780L..30C
2014ApJ...780...62L
2012ApJ...758...57L
2013ApJ...772L...2D
2014ApJ...780..123S
2013MNRAS.436....2M
2011wise.rept....1C
1992A&AS...92..481B
2014ApJ...781...20K
2014ApJ...780..140W
2014ApJ...780L...4B
2014ApJ...782...48T
2014ApJS..210...25X
2014ApJ...782...14V
2014ApJ...784...45R
2014ApJS..210...20M
2014ApJ...782...47P
2014MNRAS.439..604S
2014ApJ...788..128K
2011ApJ...743..162P
2014ApJ...791..114W
2013MNRAS.436.2956S
2013MNRAS.435.3481W
2007ApJS..168..297T
2013ApJ...771...40B
2014AJ....148...29J
2014MNRAS.443.2517H
2014Sci...345..440R
2014Sci...345...46G
2014ApJ...795...25K
2014MNRAS.443.1810B
2013ApJ...778..183L
2014MNRAS.437.1511B
2014NewA...27..102P
2013ApJ...778..184J
2014MNRAS.438.2395W
2014ApJ...783....5S


2017MNRAS.471L.125R
2017MNRAS.468L.118J
2013MNRAS.434...46H
2017AJ....154...95H
2017Natur.548...58E
2017AJ....154..135F
2012ApJ...752....1R
2017AJ....153..205D
2017MNRAS.469.1622M
2016ApJ...833L..19E
2017AJ....154..188S
2018MNRAS.473..710B
2017AJ....154..133H
2017AJ....153..256R
2017AJ....154..226D
2017AJ....154..254G
1979nlcs.book.....L
2018MNRAS.473L.131W
2017AJ....154..205M
2017AJ....154..181V
2017AJ....154..266N
2017AJ....154..194L
2018MNRAS.475.4476W
2018MNRAS.475.4467B
2018AJ....155....4M
2018AJ....155...10C
2018MNRAS.474.5523S
2017AJ....154..264T
2016AJ....152..140B
2017MNRAS.466..443J
2017AJ....153...18B
2010ApJ...723..954V
2017MNRAS.465.3693H
2016AJ....152..132G
2017MNRAS.465.3343D
2017AJ....153....9C
2014MNRAS.441.3666R
2014ApJ...794..134W
2016AJ....152..161D
2017AJ....153...51W
2017AJ....153...64M
2017MNRAS.466.2710R
2017AJ....153...94C
2017AJ....153..167W
2017PASJ...69...29N
2016MNRAS.462.1012B
2016MNRAS.463..604M
2016MNRAS.463.2922K
2016ApJ...832..191N
2017ApJ...834..151R


2013ApJ...771...11A
2014PASJ...66...94B
2015ApJ...814...67A
2014ApJ...792..112A
2017AJ....154..137J
2017MNRAS.468.2932G
2020MNRAS.491.2982E
2019AJ....158..243P
2005MNRAS.361L..15M
1998MNRAS.296..329K
2016MNRAS.457.2173G
2011IAUS..276..397B
2013MNRAS.428..182B
2014ApJ...788....2B
2015ApJ...800..138N
2018AJ....155...10C
2007AJ....133..889L
2011AJ....142..160S
2016ApJ...817...80D
2007MNRAS.376..919M
2014ApJ...790...30J
2010MNRAS.405.1140G
2006AN....327..321M
2018ApJ...866...99B
2015ApJ...799....4R
2015MNRAS.446.2959D
2012MNRAS.427.2812H
2013MNRAS.433.2097F
2014MNRAS.439.1063M
1995ApJ...443..625S
1997ApJ...476L..89P
2001AJ....122.3466M
2002ApJ...571..512H
2002ApJ...581..654P
1994AJ....107..306H
2011AJ....142..175R
2018ApJ...865...28D
2018arXiv180308163F
2013AJ....146....9A
1974AJ.....79..819H
2018MNRAS.480.2411M
2002ApJ...572L..79L
2011ApJ...726...71B
2004AN....325..718M
2006ApJ...641L..57B
2012MNRAS.421.2498G
2012ApJ...754L..20M
2013AJ....146..154M
2010PASJ...62..779N
2012PASJ...64L...7N


2022arXiv220714121A
2020PNAS..117.9712G
2022arXiv220914830G
2022arXiv220909597C
2022PASJ..tmp...97T
2022arXiv220501112B
2022arXiv220911096K
2017ApJS..231...15D
2022arXiv221010909A
2022arXiv221008996L
2022arXiv220906937B
2022ApJS..259...62I
2022AJ....164..198M
2021AJ....161....5W
2022arXiv221001027A
